# 3. Portfolio Optimization

**Objective:** To construct an optimal investment portfolio using Modern Portfolio Theory (MPT). We will integrate our forecast for TSLA with historical data for all three assets to find the allocation that provides the highest expected return for a given level of risk.

**Stakeholder Insight:** This is where data science translates directly into an investment strategy. Instead of relying on gut feeling or a static '60/40' rule, we are using a quantitative approach to build a portfolio tailored to the current market outlook, as informed by our model.

## 3.1. Setup

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import json
import sys
import os

# Add src directory to path
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

from portfolio import optimize_portfolio
from config import FORECAST_ASSET
from pypfopt import plotting

# Configure plots
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 7)

## 3.2. Load Data and Forecast

We need two key inputs:
1.  The full historical price data for all assets (to calculate risk and correlations).
2.  The forecasted annual return for TSLA (from the previous notebook).

In [ ]:
data = pd.read_csv('../data/processed/all_data.csv', index_col='Date', parse_dates=True)

with open('../reports/artifacts/forecast_return.json', 'r') as f:
    forecast_data = json.load(f)
    annual_return = forecast_data['annual_return']

print(f"Loaded forecasted annual return for TSLA: {annual_return:.2%}")

## 3.3. The Efficient Frontier

We now apply MPT to find the **Efficient Frontier**—a curve representing the set of optimal portfolios. Any portfolio on the frontier offers the highest possible expected return for its level of risk (volatility). Our goal is to find the single best portfolio on this curve.

We will target the **Maximum Sharpe Ratio Portfolio**. The Sharpe ratio measures risk-adjusted return, making it the industry standard for comparing investment strategies.

In [ ]:
weights, ef_model = optimize_portfolio(data, annual_return, FORECAST_ASSET)

## 3.4. Visualize the Optimal Portfolio

In [ ]:
# Plot the efficient frontier
fig, ax = plt.subplots()
plotting.plot_efficient_frontier(ef_model, ax=ax, show_assets=True)

# Find the tangency portfolio (Max Sharpe Ratio)
ret_tangent, std_tangent, _ = ef_model.portfolio_performance()
ax.scatter(std_tangent, ret_tangent, marker="*", s=200, c="red", label="Max Sharpe Ratio Portfolio")

plt.title('Efficient Frontier with Optimal Portfolio')
plt.legend()
plt.show()

### Stakeholder Recommendation

The analysis yields a clear, data-driven portfolio allocation:


In [ ]:
print("=== Recommended Optimal Portfolio Allocation ===")
for asset, weight in weights.items():
    print(f"{asset}: {weight:.1%}")


This allocation is designed to provide the best possible return for the level of risk taken, based on our model's forecast. It is an actionable strategy that can be implemented immediately.

The fact that the model may allocate 0% to an asset (like BND) is a powerful insight. It suggests that, given our forward-looking view on TSLA, holding bonds in this specific mix does not improve the risk-adjusted return.

## 3.5. Conclusion and Next Steps

We have successfully translated our forecast into a tangible investment strategy. The next and final step is to validate this strategy through backtesting to see how it would have performed historically.

**Action:** Save the optimal weights for the final backtesting phase.

In [ ]:
with open('../reports/artifacts/optimal_weights.json', 'w') as f:
    json.dump(weights, f)

print("Optimal portfolio weights saved to reports/artifacts/")